In [21]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [4]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text

In [5]:
with open ('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

In [ ]:
with open('../static/model/model.pickle','rb') as f:
    model = pickle.load(f)

In [6]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [8]:
def pre_processing(text):
    data = pd.DataFrame([text], columns = ['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data["tweet"].str.replace('\d+','',regex = True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [14]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens = vocab[0].tolist()

In [15]:
def vectorizer(ds,vocabulary):
    vectorized_lst = []

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range (len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1
        vectorized_lst.append(sentence_list)
    vectorized_lst_new = np.asarray(vectorized_lst,dtype = np.float32)
    return vectorized_lst_new

In [32]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return "Negative"
    else:
        return "Positive"

In [34]:
txt = "good"
pre_processed_text = pre_processing(txt)
vectorized_text = vectorizer(pre_processed_text,tokens)
prediction = get_prediction(vectorized_text)
prediction

'Negative'